This notebook demonstrates how to use `yamlpyowl` and `owlready2` analyze and solve the famous [zebra puzzle](http://en.wikipedia.org/wiki/Zebra_puzzle) by A. Einstein. It consists of 15 clues like:

1. There are five houses.
2. The Englishman lives in the red house.
3. The Spaniard owns the dog.
4. Coffee is drunk in the green house.
5. ...


Here you can view the source file ([YAML](https://en.wikipedia.org/wiki/YAML)-Syntax): [examples/einsteins_zebra_riddle.owl.yml](../../examples/einsteins_zebra_riddle.owl.yml)

Note: The OWL-representation is based on the work of [D. Ponomaryov](https://persons.iis.nsk.su/en/ponom/ontologies)


In [1]:
import yamlpyowl as ypo

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
fpath = "../../examples/einsteins_zebra_riddle.owl.yml"
om = ypo.OntologyManager(fpath)

# create an object for quick access to all entities:
n = om.name_mapping_container

### Investigate which information is represented in the ontology itself (without running the reasoner)

In [3]:
# note: not a `Pet` but still a `Thing`
n.dog

<Thing 'dog'>

In [4]:
n.house_1.left_to

<Thing 'house_2'>

In [5]:
n.house_1.right_to

<<property object at 0x7fb07d64d630> 'Nothing'>

In [6]:
n.dog == n.Spaniard.owns

False

In [7]:
# This is the owlready-Ontology
om.onto

get_ontology("https://w3id.org/yet/undefined/einstein-zebra-puzzle-ontology#")

In [8]:
print("classes (concepts):\n", list(om.onto.classes()), "\n"*2)
print("properties (roles):\n", list(om.onto.properties()), "\n"*2)

classes (concepts):
 [<<property object at 0x7fb07d64d630> 'Pet'>, <<property object at 0x7fb07d64d630> 'Man'>, <<property object at 0x7fb07d64d630> 'House'>, <<property object at 0x7fb07d64d630> 'Color'>, <<property object at 0x7fb07d64d630> 'Beverage'>, <<property object at 0x7fb07d64d630> 'Cigarette'>] 


properties (roles):
 [<<property object at 0x7fb07d64d630> 'right_to'>, <<property object at 0x7fb07d64d630> 'left_to'>, <<property object at 0x7fb07d64d630> 'has_color'>, <<property object at 0x7fb07d64d630> 'inv_has_color'>, <<property object at 0x7fb07d64d630> 'drinks'>, <<property object at 0x7fb07d64d630> 'inv_drinks'>, <<property object at 0x7fb07d64d630> 'lives_in'>, <<property object at 0x7fb07d64d630> 'inv_lives_in'>, <<property object at 0x7fb07d64d630> 'owns'>, <<property object at 0x7fb07d64d630> 'inv_owns'>, <<property object at 0x7fb07d64d630> 'smokes'>, <<property object at 0x7fb07d64d630> 'inv_smokes'>] 




In [9]:
list(om.onto.lives_in.get_relations())

[(<Thing 'Norwegian'>, <Thing 'house_1'>)]

In [10]:
# the axioms object contains important pieces of information
# (but does not cotain "real" axioms due to missing subject, to which restriction this axiom-fragment belongs)

axioms = list(om.onto.general_axioms())
print(f"{len(axioms)=}", "\n")
print("\n\n".join(map(str, axioms[:3])))
axioms[-2:]

len(axioms)=37 

<<property object at 0x7fb07d64d630> 'left_to'>.some(<<property object at 0x7fb07d64d630> 'has_color'>.value(<Thing 'blue'>))

<<property object at 0x7fb07d64d630> 'has_color'>.value(<Thing 'blue'>)

<<property object at 0x7fb07d64d630> 'lives_in'>.some(<<property object at 0x7fb07d64d630> 'left_to'>.some(<<property object at 0x7fb07d64d630> 'has_color'>.value(<Thing 'blue'>)))


[OneOf([<Thing 'Englishman'>, <Thing 'Japanese'>, <Thing 'Norwegian'>, <Thing 'Spaniard'>, <Thing 'Ukrainian'>]),
 OneOf([<Thing 'dog'>, <Thing 'zebra'>, <Thing 'snails'>, <Thing 'fox'>, <Thing 'horse'>])]

### Call the reasoner and investigate the entities again

In [11]:
%time om.sync_reasoner(infer_property_values=True)

CPU times: user 21.3 ms, sys: 371 µs, total: 21.6 ms
Wall time: 6.2 s


The puzzle is called "zebra puzzle" because the question is: **Who owns the zebra?** This can be answered with a very simple [SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial) query.

In [12]:
qsrc = f"""
PREFIX P: <{om.iri}>
SELECT ?x WHERE {{
?x P:owns P:zebra.
}}
"""
om.make_query(qsrc)

{<Man 'Japanese'>}

Of couse, the entities can be accessed also directly:

In [13]:
list(om.onto.lives_in.get_relations())

[(<Man 'Norwegian'>, <House 'house_1'>),
 (<Man 'Englishman'>, <House 'house_3'>),
 (<Man 'Japanese'>, <House 'house_5'>),
 (<Man 'Spaniard'>, <House 'house_4'>),
 (<Man 'Ukrainian'>, <House 'house_2'>),
 (<Man 'Ukrainian'>, <House 'house_2'>),
 (<Man 'Spaniard'>, <House 'house_4'>),
 (<Man 'Japanese'>, <House 'house_5'>),
 (<Man 'Englishman'>, <House 'house_3'>)]

In [14]:
list(om.onto.owns.get_relations())

[(<Man 'Englishman'>, <Pet 'snails'>),
 (<Man 'Norwegian'>, <Pet 'fox'>),
 (<Man 'Japanese'>, <Pet 'zebra'>),
 (<Man 'Spaniard'>, <Pet 'dog'>),
 (<Man 'Ukrainian'>, <Pet 'horse'>),
 (<Man 'Japanese'>, <Pet 'zebra'>),
 (<Man 'Norwegian'>, <Pet 'fox'>),
 (<Man 'Englishman'>, <Pet 'snails'>),
 (<Man 'Spaniard'>, <Pet 'dog'>),
 (<Man 'Ukrainian'>, <Pet 'horse'>)]

In [15]:
list(om.onto.drinks.get_relations())

[(<Man 'Ukrainian'>, <Beverage 'tea'>),
 (<Man 'Englishman'>, <Beverage 'milk'>),
 (<Man 'Norwegian'>, <Beverage 'water'>),
 (<Man 'Japanese'>, <Beverage 'coffee'>),
 (<Man 'Spaniard'>, <Beverage 'orange_juice'>),
 (<Man 'Englishman'>, <Beverage 'milk'>),
 (<Man 'Norwegian'>, <Beverage 'water'>),
 (<Man 'Japanese'>, <Beverage 'coffee'>),
 (<Man 'Spaniard'>, <Beverage 'orange_juice'>)]

### Concluding Hypothesis
Logical puzzles have significant didactic potential to increase the accessibility of semantic technologies.